In [ ]:
from keras.models import model_from_yaml
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
from tqdm import tqdm_notebook
from glob import glob
% matplotlib inline


In [ ]:
with open("ModelArchive/u-net-model.yaml") as f:
    model = model_from_yaml(f.read())
model.load_weights("ModelArchive/release_v1.0.hdf5")


In [ ]:
from utils.keras_tools import ImageDirectoryIterator, x_tif_reader, y_tif_reader, load_from_directories
import re, os

In [ ]:
def save_tif(output, array):
    driver = gdal.GetDriverByName('GTiff')
    new_tif = driver.Create(output, 256, 256, 1, gdal.GDT_Byte)
    new_tif.GetRasterBand(1).WriteArray(array)


In [ ]:
for row in tqdm_notebook(range(109799, 110029), desc=u"预测"):
    filepaths = glob("/Users/mac/Desktop/dest/{}_*.jpeg".format(row))
    gen = ImageDirectoryIterator([(filepath, None) for filepath in filepaths], x_tif_reader, None, batch_size=16,
                                 shuffle=False)
    predic_y = model.predict_generator(gen, steps=len(gen), verbose=0)
    predic_y_byte = (predic_y * 255).astype(np.uint8)
    for i, filepath in enumerate(filepaths):
        filename = os.path.basename(filepath)
        gray = predic_y_byte[i].reshape(256, 256)
        save_tif(os.path.join("/Users/mac/Desktop/model_prediction", os.path.splitext(filename)[0] + ".tif"), gray)
